In [12]:
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import pandas as pd

In [2]:
file_name = 'mfcc_aug_flattened.csv'

In [3]:
df = pd.read_csv(file_name)

X = df.drop('diagnosis', axis=1)  # Features
y = df['diagnosis']

In [4]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [5]:
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_scaled)

In [6]:
X_pca.shape

(3680, 1908)

In [7]:
y.shape

(3680,)

In [8]:
X_train, X_temp, y_train, y_temp = train_test_split(X_pca, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [9]:
def k_fold_cross_validation(model, X, y, k=5):
    scores = cross_val_score(model, X, y, cv=k, scoring='accuracy')
    return scores

In [10]:
models = {
    'SVM': SVC(),
    'KNN': KNeighborsClassifier(),
    'LR': LogisticRegression(),
    'Random Forest': RandomForestClassifier(),
    'Naive Bayes': GaussianNB(),
}

In [13]:
results = {}

for name, model in tqdm(models.items()):
    model.fit(X_train, y_train)

    predictions = model.predict(X_test)

    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions,average='weighted')
    recall = recall_score(y_test, predictions,average='weighted')
    f1 = f1_score(y_test, predictions,average='weighted')

    cv_scores = k_fold_cross_validation(model, X_pca, y)

    results[name] = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Cross-Validation Scores': cv_scores,
    }

  0%|          | 0/5 [00:00<?, ?it/s]c:\Users\mercu\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mercu\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
 20%|██        | 1/5 [01:11<04:45, 71.39s/it]c:\Users\mercu\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mercu\AppData\Local\P

In [14]:
for name, metrics in results.items():
    print(f"Model: {name}")
    print(f"  Accuracy: {metrics['Accuracy']:.4f}")
    print(f"  Precision: {metrics['Precision']:.4f}")
    print(f"  Recall: {metrics['Recall']:.4f}")
    print(f"  F1 Score: {metrics['F1 Score']:.4f}")
    print(f"  Mean Cross-Validation Accuracy: {np.mean(metrics['Cross-Validation Scores']):.4f}")
    print("")

Model: SVM
  Accuracy: 0.8732
  Precision: 0.8547
  Recall: 0.8732
  F1 Score: 0.8274
  Mean Cross-Validation Accuracy: 0.8845

Model: KNN
  Accuracy: 0.8659
  Precision: 0.8202
  Recall: 0.8659
  F1 Score: 0.8197
  Mean Cross-Validation Accuracy: 0.8793

Model: LR
  Accuracy: 0.6993
  Precision: 0.8728
  Recall: 0.6993
  F1 Score: 0.7550
  Mean Cross-Validation Accuracy: 0.6329

Model: Random Forest
  Accuracy: 0.8551
  Precision: 0.7710
  Recall: 0.8551
  F1 Score: 0.7899
  Mean Cross-Validation Accuracy: 0.8668

Model: Naive Bayes
  Accuracy: 0.5000
  Precision: 0.8271
  Recall: 0.5000
  F1 Score: 0.5952
  Mean Cross-Validation Accuracy: 0.4296

